In [1]:
from worker import Worker
import tensorflow as tf



In [2]:
coordinator = tf.train.Coordinator()

for idx in range(2):
    t = Worker(args=idx, kwargs={'coordinator': coordinator, 'hidden_layer_size': 128, 'dropout_rate': 1-0.2, 'batch_size': 128, 'epochs': 25})
    t.start()


NameError: name 'tf' is not defined

In [2]:
class ResNet(tf.keras.Model):

    def __init__(self):
        super(ResNet, self).__init__()
        self.block_1 = ResNetBlock()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)


resnet = ResNet()


NameError: name 'ResNetBlock' is not defined